# Model Test Notebook
## Halil Melih AKÇA 221104091

Bu notebook, eğitilmiş modelin `Combined_News_DJIA_table.csv` verileri ile performans testini yapar.

## Gerekli Kütüphaneleri Import Et

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import string
import warnings
warnings.filterwarnings('ignore')

# Machine Learning libraries
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# NLP libraries
try:
    import nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()
    print("✓ VADER sentiment analyzer loaded")
except:
    print("✗ VADER not available")
    sia = None

try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
    print("✓ spaCy model loaded")
except:
    print("✗ spaCy model not available")
    nlp = None

try:
    from textstat import flesch_reading_ease, automated_readability_index
    print("✓ textstat library loaded")
except:
    print("✗ textstat not available")
    flesch_reading_ease = lambda x: 0
    automated_readability_index = lambda x: 0

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')

print("\n✅ Kütüphaneler yüklendi!")

✓ VADER sentiment analyzer loaded
✓ spaCy model loaded
✓ textstat library loaded

✅ Kütüphaneler yüklendi!


## Veri Yükleme ve Model Yükleme

In [ ]:
# Test verilerini yükle
try:
    # İlk önce Combined_News_DJIA.csv'yi dene (orijinal)
    test_df = pd.read_csv("../stockMarket_predict/Combined_News_DJIA.csv")
    print(f"✓ Original dataset loaded: {test_df.shape}")
    data_source = "Combined_News_DJIA.csv"
except FileNotFoundError:
    try:
        # Sonra upload_DJIA_table.csv'yi dene
        test_df = pd.read_csv("../stockMarket_predict/upload_DJIA_table.csv")
        print(f"✓ Upload dataset loaded: {test_df.shape}")
        data_source = "upload_DJIA_table.csv"
    except FileNotFoundError:
        print("✗ Hiçbir veri dosyası bulunamadı!")
        exit()

print(f"\nVeri kaynağı: {data_source}")
print(f"Veri boyutu: {test_df.shape}")
print(f"Sütunlar: {list(test_df.columns)}")

# Eğitilmiş modeli yükle
try:
    with open("ensemble_model.pkl", "rb") as f:
        trained_model = pickle.load(f)
    print("\n✓ Eğitilmiş ensemble model yüklendi")
    print(f"Model tipi: {type(trained_model).__name__}")
    if hasattr(trained_model, 'estimators'):
        print(f"Base estimators: {len(trained_model.estimators)}")
        for name, estimator in trained_model.estimators:
            print(f"  - {name}: {type(estimator).__name__}")
except FileNotFoundError:
    print("\n✗ Model dosyası bulunamadı! Önce training notebook'u çalıştırın.")
    trained_model = None

✗ Hiçbir veri dosyası bulunamadı!


NameError: name 'data_source' is not defined

: 

## Feature Engineering Fonksiyonları

In [ ]:
# Orijinal notebook'taki feature engineering fonksiyonları

def clean_text(text):
    """Metin temizleme fonksiyonu"""
    if pd.isna(text):
        return ""
    
    text = str(text).lower()
    text = re.sub(r"b['\"]|['\"]", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\d+', 'NUMBER', text)
    text = re.sub(r'[^\w\s.,!?;:]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def linguistic_features(text):
    """Linguistik özellikler çıkarma"""
    words = text.split()
    avg_word_len = np.mean([len(w) for w in words]) if words else 0
    punct_count = sum([1 for c in text if c in string.punctuation])
    cap_ratio = sum([1 for c in text if c.isupper()]) / (len(text) + 1e-9)
    digit_ratio = sum([1 for c in text if c.isdigit()]) / (len(text) + 1e-9)
    
    try:
        flesch = flesch_reading_ease(text)
        ari = automated_readability_index(text)
    except:
        flesch = 0
        ari = 0
    
    return [len(words), avg_word_len, punct_count, cap_ratio, digit_ratio, flesch, ari]

def pos_features_spacy(text):
    """POS tag özellikleri (spaCy ile)"""
    if nlp is None:
        return [0.25, 0.25, 0.25, 0.25]  # Default değerler
    
    try:
        doc = nlp(text)
        total = len(doc)
        if total == 0:
            return [0, 0, 0, 0]
        noun_ratio = len([token for token in doc if token.pos_ == "NOUN"]) / total
        verb_ratio = len([token for token in doc if token.pos_ == "VERB"]) / total
        adj_ratio = len([token for token in doc if token.pos_ == "ADJ"]) / total
        adv_ratio = len([token for token in doc if token.pos_ == "ADV"]) / total
        return [noun_ratio, verb_ratio, adj_ratio, adv_ratio]
    except:
        return [0.25, 0.25, 0.25, 0.25]

financial_keywords = ["bull", "bear", "gain", "loss", "stock", "market"]

def financial_keyword_density(text):
    """Finansal anahtar kelime yoğunluğu"""
    tokens = text.lower().split()
    return [tokens.count(word)/len(tokens) if len(tokens) > 0 else 0 for word in financial_keywords]

def ner_features(text):
    """Named Entity Recognition özellikleri"""
    if nlp is None:
        return [0, 0, 0, 0]  # Default değerler
    
    try:
        doc = nlp(text)
        counts = {"PERSON":0, "ORG":0, "GPE":0, "MONEY":0}
        for ent in doc.ents:
            if ent.label_ in counts:
                counts[ent.label_] += 1
        return list(counts.values())
    except:
        return [0, 0, 0, 0]

print("✅ Feature engineering fonksiyonları tanımlandı!")

## Veri Ön İşleme

In [ ]:
# Veriyi hazırla
print("=== VERİ ÖN İŞLEME ===")

# Haberleri birleştir (2-27 arası sütunlar)
if test_df.shape[1] >= 27:
    test_df['Combined'] = test_df.iloc[:, 2:27].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
else:
    # Eğer sütun sayısı az ise, mevcut tüm haber sütunlarını kullan
    news_columns = [col for col in test_df.columns if col not in ['Date', 'Label']]
    test_df['Combined'] = test_df[news_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Metinleri temizle
test_df['Cleaned'] = test_df['Combined'].apply(clean_text)

print(f"✓ {len(test_df)} örnek işlendi")
print(f"✓ Ortalama metin uzunluğu: {test_df['Cleaned'].str.len().mean():.1f} karakter")

# Label kontrolü
if 'Label' in test_df.columns:
    print(f"\n📊 Label Dağılımı:")
    label_counts = test_df['Label'].value_counts()
    for label, count in label_counts.items():
        percentage = (count / len(test_df)) * 100
        print(f"  Label {label}: {count} örnek ({percentage:.1f}%)")
else:
    print("⚠️ Label sütunu bulunamadı - sadece tahmin yapılacak")

# İlk birkaç örneği göster
print("\n📝 İlk 3 örnek:")
for i in range(min(3, len(test_df))):
    text = test_df['Cleaned'].iloc[i][:100] + "..." if len(test_df['Cleaned'].iloc[i]) > 100 else test_df['Cleaned'].iloc[i]
    label = test_df['Label'].iloc[i] if 'Label' in test_df.columns else "?"
    print(f"  {i+1}. Label: {label} | Text: {text}")

## Feature Extraction

In [ ]:
print("=== FEATURE EXTRACTION ===")

# 1. Linguistik özellikler
print("📊 Linguistik özellikler çıkarılıyor...")
ling_features = test_df['Cleaned'].apply(linguistic_features)
ling_df = pd.DataFrame(ling_features.tolist(), 
                      columns=["word_count", "avg_word_len", "punct_count", 
                              "cap_ratio", "digit_ratio", "flesch", "ari"])
print(f"  ✓ {ling_df.shape[1]} linguistik özellik çıkarıldı")

# 2. Sentiment özellikleri
print("😊 Sentiment özellikleri çıkarılıyor...")
if sia is not None:
    sentiment_features = test_df['Cleaned'].apply(lambda x: pd.Series(sia.polarity_scores(x)))
    print(f"  ✓ {sentiment_features.shape[1]} sentiment özellik çıkarıldı")
else:
    sentiment_features = pd.DataFrame({
        'neg': [0] * len(test_df), 'neu': [0.5] * len(test_df), 
        'pos': [0] * len(test_df), 'compound': [0] * len(test_df)
    })
    print("  ⚠️ VADER kullanılamadı, default değerler kullanıldı")

# 3. POS özellikleri
print("🏷️ POS tag özellikleri çıkarılıyor...")
pos_features = test_df['Cleaned'].apply(pos_features_spacy)
pos_df = pd.DataFrame(pos_features.tolist(), 
                     columns=["noun_ratio", "verb_ratio", "adj_ratio", "adv_ratio"])
print(f"  ✓ {pos_df.shape[1]} POS özellik çıkarıldı")

# 4. Finansal anahtar kelime özellikleri
print("💰 Finansal anahtar kelime özellikleri çıkarılıyor...")
fin_features = test_df['Cleaned'].apply(financial_keyword_density)
fin_df = pd.DataFrame(fin_features.tolist(), 
                     columns=[f'kw_{k}' for k in financial_keywords])
print(f"  ✓ {fin_df.shape[1]} finansal özellik çıkarıldı")

# 5. NER özellikleri
print("🏢 Named Entity özellikleri çıkarılıyor...")
ner_features = test_df['Cleaned'].apply(ner_features)
ner_df = pd.DataFrame(ner_features.tolist(), 
                     columns=["PERSON", "ORG", "GPE", "MONEY"])
print(f"  ✓ {ner_df.shape[1]} NER özellik çıkarıldı")

# 6. TF-IDF özellikleri (PCA ile boyut azaltma)
print("📝 TF-IDF özellikleri çıkarılıyor...")
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2), 
                       min_df=2, max_df=0.95, stop_words='english')
tfidf_matrix = tfidf.fit_transform(test_df['Cleaned'])
pca = PCA(n_components=50)
tfidf_pca = pca.fit_transform(tfidf_matrix.toarray())
tfidf_df = pd.DataFrame(tfidf_pca, columns=[f'pca_{i}' for i in range(tfidf_pca.shape[1])])
print(f"  ✓ {tfidf_df.shape[1]} TF-IDF+PCA özellik çıkarıldı")

# Tüm özellikleri birleştir
all_features = pd.concat([ling_df, sentiment_features, pos_df, fin_df, ner_df, tfidf_df], axis=1)

print(f"\n✅ Toplam {all_features.shape[1]} özellik çıkarıldı!")
print(f"📊 Feature matrix boyutu: {all_features.shape}")

# Feature istatistikleri
print("\n📈 Feature İstatistikleri:")
print(f"  - Linguistik: {ling_df.shape[1]} özellik")
print(f"  - Sentiment: {sentiment_features.shape[1]} özellik")
print(f"  - POS Tags: {pos_df.shape[1]} özellik")
print(f"  - Finansal: {fin_df.shape[1]} özellik")
print(f"  - NER: {ner_df.shape[1]} özellik")
print(f"  - TF-IDF+PCA: {tfidf_df.shape[1]} özellik")

## Feature Scaling ve Polynomial Features

In [ ]:
print("=== FEATURE SCALING VE TRANSFORMATION ===")

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(all_features.values)
print(f"✓ Feature scaling tamamlandı")

# Polynomial features (eğitim sırasında kullanıldıysa)
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_poly = poly.fit_transform(X_scaled)
print(f"✓ Polynomial features oluşturuldu")
print(f"📊 Final feature matrix boyutu: {X_poly.shape}")

# Memory kullanımı kontrolü
memory_usage_mb = X_poly.nbytes / (1024 * 1024)
print(f"💾 Memory kullanımı: {memory_usage_mb:.1f} MB")

if memory_usage_mb > 1000:
    print("⚠️ Yüksek memory kullanımı tespit edildi!")
else:
    print("✅ Memory kullanımı kabul edilebilir seviyede")

## Model Test ve Tahmin

In [ ]:
print("=== MODEL TEST VE TAHMİN ===")

if trained_model is None:
    print("❌ Model yüklenmediği için tahmin yapılamıyor!")
else:
    try:
        # Model ile tahmin yap
        print("🤖 Model ile tahminler yapılıyor...")
        y_pred = trained_model.predict(X_poly)
        
        # Prediction probabilities (eğer mevcut ise)
        try:
            y_pred_proba = trained_model.predict_proba(X_poly)
            print("✓ Tahmin olasılıkları da hesaplandı")
        except:
            y_pred_proba = None
            print("⚠️ Model predict_proba desteklemiyor")
        
        print(f"✅ {len(y_pred)} tahmin başarıyla yapıldı!")
        
        # Tahmin dağılımı
        print("\n📊 Tahmin Dağılımı:")
        pred_counts = pd.Series(y_pred).value_counts().sort_index()
        for pred, count in pred_counts.items():
            percentage = (count / len(y_pred)) * 100
            print(f"  Prediction {pred}: {count} örnek ({percentage:.1f}%)")
        
        # Eğer gerçek labellar varsa, performans metriklerini hesapla
        if 'Label' in test_df.columns:
            y_true = test_df['Label'].values
            
            print("\n🎯 PERFORMANS METRİKLERİ:")
            
            # Accuracy
            accuracy = accuracy_score(y_true, y_pred)
            print(f"📈 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
            
            # F1 Score
            f1 = f1_score(y_true, y_pred)
            print(f"📈 F1 Score: {f1:.4f}")
            
            # Classification Report
            print("\n📋 Detaylı Classification Report:")
            print(classification_report(y_true, y_pred))
            
            # Confusion Matrix
            print("\n🧩 Confusion Matrix:")
            cm = confusion_matrix(y_true, y_pred)
            print(cm)
            
            # Confusion Matrix Visualization
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                       xticklabels=['Down (0)', 'Up (1)'], 
                       yticklabels=['Down (0)', 'Up (1)'])
            plt.title('Confusion Matrix')
            plt.ylabel('Gerçek Label')
            plt.xlabel('Tahmin Edilen Label')
            plt.show()
            
            # Baseline ile karşılaştırma
            baseline_accuracy = max(pd.Series(y_true).value_counts(normalize=True))
            print(f"\n📊 Baseline Accuracy (Majority Class): {baseline_accuracy:.4f}")
            improvement = accuracy - baseline_accuracy
            print(f"📈 Model İyileştirmesi: {improvement:.4f} ({improvement*100:.2f} percentage points)")
            
        else:
            print("\n⚠️ Gerçek labellar olmadığı için performans metrikleri hesaplanamıyor")
            print("Sadece tahminler yapıldı.")
        
    except Exception as e:
        print(f"❌ Tahmin sırasında hata oluştu: {e}")
        import traceback
        traceback.print_exc()

## Tahmin Örnekleri ve Analiz

In [ ]:
if trained_model is not None and 'y_pred' in locals():
    print("=== TAHMİN ÖRNEKLERİ VE ANALİZ ===")
    
    # Tahmin sonuçlarını dataframe'e ekle
    results_df = test_df.copy()
    results_df['Prediction'] = y_pred
    
    if y_pred_proba is not None:
        results_df['Prob_Down'] = y_pred_proba[:, 0]
        results_df['Prob_Up'] = y_pred_proba[:, 1]
        results_df['Confidence'] = np.max(y_pred_proba, axis=1)
    
    # İlk 10 örneği göster
    print("\n📝 İlk 10 Tahmin Örneği:")
    print("-" * 100)
    
    for i in range(min(10, len(results_df))):
        text = results_df['Cleaned'].iloc[i][:80] + "..." if len(results_df['Cleaned'].iloc[i]) > 80 else results_df['Cleaned'].iloc[i]
        pred = results_df['Prediction'].iloc[i]
        pred_text = "📈 UP" if pred == 1 else "📉 DOWN"
        
        if 'Label' in results_df.columns:
            true_label = results_df['Label'].iloc[i]
            true_text = "📈 UP" if true_label == 1 else "📉 DOWN"
            correct = "✅" if pred == true_label else "❌"
            
            if 'Confidence' in results_df.columns:
                confidence = results_df['Confidence'].iloc[i]
                print(f"{i+1:2d}. {correct} Gerçek: {true_text} | Tahmin: {pred_text} | Güven: {confidence:.3f}")
            else:
                print(f"{i+1:2d}. {correct} Gerçek: {true_text} | Tahmin: {pred_text}")
        else:
            if 'Confidence' in results_df.columns:
                confidence = results_df['Confidence'].iloc[i]
                print(f"{i+1:2d}. Tahmin: {pred_text} | Güven: {confidence:.3f}")
            else:
                print(f"{i+1:2d}. Tahmin: {pred_text}")
        
        print(f"    Text: {text}")
        print()
    
    # Güven düzeyi analizi (eğer varsa)
    if 'Confidence' in results_df.columns:
        print("\n📊 Güven Düzeyi Analizi:")
        print(f"  Ortalama güven: {results_df['Confidence'].mean():.3f}")
        print(f"  Minimum güven: {results_df['Confidence'].min():.3f}")
        print(f"  Maksimum güven: {results_df['Confidence'].max():.3f}")
        
        # Güven düzeyi histogramı
        plt.figure(figsize=(10, 6))
        plt.hist(results_df['Confidence'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        plt.title('Model Güven Düzeyi Dağılımı')
        plt.xlabel('Güven Düzeyi')
        plt.ylabel('Frekans')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Yüksek ve düşük güvenli tahminler
        high_conf = results_df[results_df['Confidence'] > 0.8]
        low_conf = results_df[results_df['Confidence'] < 0.6]
        
        print(f"\n🎯 Yüksek güvenli tahminler (>0.8): {len(high_conf)} örnek")
        print(f"🤔 Düşük güvenli tahminler (<0.6): {len(low_conf)} örnek")
        
        if 'Label' in results_df.columns and len(high_conf) > 0:
            high_conf_accuracy = accuracy_score(high_conf['Label'], high_conf['Prediction'])
            print(f"📈 Yüksek güvenli tahminlerde accuracy: {high_conf_accuracy:.4f}")

else:
    print("⚠️ Model tahminleri mevcut değil, analiz yapılamıyor.")

## Model Performance Summary

In [ ]:
print("=" * 80)
print("                    MODEL TEST RAPORU")
print("=" * 80)

print(f"\n📊 TEST VERİSİ BİLGİLERİ:")
print(f"  📁 Veri kaynağı: {data_source}")
print(f"  📈 Toplam örnek sayısı: {len(test_df)}")
print(f"  🔧 Çıkarılan feature sayısı: {all_features.shape[1]}")
print(f"  ⚙️ Final feature matrix: {X_poly.shape}")

if 'Label' in test_df.columns:
    print(f"\n🎯 LABEL BİLGİLERİ:")
    label_counts = test_df['Label'].value_counts().sort_index()
    for label, count in label_counts.items():
        percentage = (count / len(test_df)) * 100
        direction = "📉 DOWN" if label == 0 else "📈 UP"
        print(f"  {direction}: {count} örnek ({percentage:.1f}%)")

if trained_model is not None:
    print(f"\n🤖 MODEL BİLGİLERİ:")
    print(f"  🔧 Model tipi: {type(trained_model).__name__}")
    if hasattr(trained_model, 'estimators'):
        print(f"  👥 Base estimator sayısı: {len(trained_model.estimators)}")
        for name, estimator in trained_model.estimators:
            print(f"     - {name}: {type(estimator).__name__}")
    
    if 'y_pred' in locals() and 'Label' in test_df.columns:
        print(f"\n📈 PERFORMANS METRİKLERİ:")
        print(f"  🎯 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"  🎯 F1 Score: {f1:.4f}")
        print(f"  📊 Baseline Accuracy: {baseline_accuracy:.4f}")
        print(f"  📈 Model İyileştirmesi: +{improvement:.4f} ({improvement*100:.2f} pp)")
        
        # Performans değerlendirmesi
        if accuracy > 0.55:
            print(f"  ✅ İyi performans! (>55%)")
        elif accuracy > 0.50:
            print(f"  ⚠️ Orta performans (50-55%)")
        else:
            print(f"  ❌ Düşük performans (<50%)")
            
        if improvement > 0.05:
            print(f"  ✅ Baseline'dan anlamlı iyileştirme!")
        else:
            print(f"  ⚠️ Baseline'dan sınırlı iyileştirme")
else:
    print(f"\n❌ Model yüklenemedi - performans testi yapılamadı")

print(f"\n💡 FİNANSAL SENTIMENT ANALİZ NOTLARI:")
print(f"  • Haber sentiment analizi tipik olarak %45-60 accuracy gösterir")
print(f"  • Haber verileri piyasa hareketlerinin sadece ~%3'ünü açıklar")
print(f"  • %53 civarı accuracy finansal ML'de normal kabul edilir")
print(f"  • Yüksek accuracy overfitting işareti olabilir")

print(f"\n⏰ Test tarihi: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"👤 Test eden: Halil Melih AKÇA (221104091)")
print("=" * 80)
print("✅ MODEL TEST TAMAMLANDI!")
print("=" * 80)